<a href="https://colab.research.google.com/github/VaibhavdLights/Data-Extraction-EMG/blob/main/Honors_DataExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
folder_path = '/content/drive/MyDrive/Vaibhav Honors Data'
!ls "$folder_path"

'Dummy EMG Data'  'Subject 1'	'Subject 11'  'Subject 2'  'Subject 4'	'Subject 6'  'Subject 8'
'Setup Pictures'  'Subject 10'	'Subject 12'  'Subject 3'  'Subject 5'	'Subject 7'  'Subject 9'


In [ ]:
import os

In [ ]:
subjects = os.listdir(folder_path)

In [ ]:
subjects = [i for i in subjects if i.startswith('Subject')]

In [ ]:
subjects

['Subject 1',
 'Subject 2',
 'Subject 3',
 'Subject 4',
 'Subject 5',
 'Subject 6',
 'Subject 7',
 'Subject 8',
 'Subject 9',
 'Subject 10',
 'Subject 11',
 'Subject 12']

In [ ]:
os.listdir(os.path.join(folder_path, subjects[0]))

['info.txt', 'LeapMotion Data', 'Scores', 'EMG Data']

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define the root folder path
root_folder_path = '/content/drive/MyDrive/Vaibhav Honors Data'

# Define the EMG data folder name
emg_data_path = 'EMG Data'

# Define the target base folder for reorganized data
target_base_folder = '/content/drive/MyDrive/Reorganized_Data'

In [ ]:
def data_extraction(file, file_name):
    df1 = pd.read_csv(file).iloc[2:]
    df1 = df1.reset_index(drop='first')
    df1.drop(columns=['type', 'time_units', 'begin_time', 'frequency', 'count', 'units'], inplace=True)
    df1.rename(columns={'name':file_name.split('-')[1].split(' ')[0]},inplace=True)
    df1[file_name.split('-')[1].split(' ')[0]] = df1[file_name.split('-')[1].split(' ')[0]].astype('object').astype('float')
    return df1

In [ ]:
dfs = []
file_names = []

for subject in subjects:
    folder_path = os.path.join(root_folder_path, subject, emg_data_path)

    # List files excluding 'info.csv' and sort them
    files = [file_name for file_name in os.listdir(folder_path) if file_name != 'info.csv']
    sorted_files = sorted(files, key=lambda x: int(x.split('_')[0]))

    for file_name in sorted_files:
        file_path = os.path.join(folder_path, file_name)
        df = data_extraction(file_path, file_name)
        dfs.append(df)
        file_names.append(file_name)

In [ ]:
dataframes = []

for i in range(len(dfs)//5):
    new_df = pd.concat([dfs[i*5], dfs[i*5+1], dfs[i*5+2], dfs[i*5+3], dfs[i*5+4]], axis=1)
    dataframes.append(new_df)

In [ ]:
dataframes[0]

,ED,ECR,ECU,APL,FDI
0,36.005314,-3.942514,-17.715765,0.612067,5.184034
1,37.531286,3.992541,-18.936543,1.222455,7.015201
2,37.226092,11.927595,-14.663821,2.443233,9.151562
3,59.200089,20.167844,-9.780711,4.274399,10.067145
4,88.803947,26.271732,-7.644350,4.884788,9.456756
...,...,...,...,...,...
89835,29.596231,12.537984,-28.397569,0.001678,-87.900260
89836,-19.234874,21.083428,-15.274210,2.138039,-216.387106
89837,-59.520536,21.388622,4.868621,2.138039,-126.049561
89838,-76.306228,11.622401,11.888092,2.138039,56.151500


In [ ]:
concatenated_df = pd.concat([dataframes[283], dataframes[284]], axis=0)

dataframes[283] = concatenated_df

dataframes.pop(284)

,ED,ECU,APL,FDI,ECR
0,-6.352497,-0.291468,82.335838,39.872940,-19.840992
1,-9.404441,-2.427828,95.154004,52.080716,-13.737104
2,-11.845997,-4.564189,121.095528,66.119659,-6.412438
3,-10.320025,-6.090161,152.835747,79.853408,1.522617
4,-4.521331,-7.921328,176.030522,91.145601,10.068060
...,...,...,...,...,...
86915,-6.047303,3.370865,-60.189950,-28.490607,-22.587742
86916,-4.826525,2.455282,-41.878286,-23.912691,-19.535798
86917,-3.910942,2.455282,-11.969234,-18.419192,-17.704631
86918,-2.690164,1.844893,23.738512,-9.873748,-14.347493


In [ ]:
folder_path = target_base_folder

for i, df in enumerate(dataframes):
    subject_num = i // (2 * 2 * 4 * 5)
    weapon_num = (i // (2 * 4 * 5)) % 2
    mode_num = (i // (4 * 5)) % 2
    mouse_num = (i // 5) % 4
    trial_num = i % 5 + 1

    subject_folder = os.path.join(folder_path, f"Subject{subject_num+1}")
    weapon_folder = os.path.join(subject_folder, "Vandal" if weapon_num == 0 else "Guardian")
    mode_folder = os.path.join(weapon_folder, "Hard" if mode_num == 0 else "Medium")
    mouse_folder = os.path.join(mode_folder, f"Mouse{mouse_num+1}")

    for folder in [subject_folder, weapon_folder, mode_folder, mouse_folder]:
        os.makedirs(folder, exist_ok=True)

    trial_file = os.path.join(mouse_folder, f"Trial{trial_num}.csv")
    df.to_csv(trial_file, index=False)

print("Data organization completed.")

Data organization completed.


In [ ]:
len(dataframes)

960